### Group Case Study: SA01 - TEAM07
 - [Data Details](#id01)
 - [Questions to Answer](#id02)
     - Q2
     - Q3

### <a id="id01"/> DataDetails
 ###### To be added 

### <a id="id02"/>Questions to answer:
 1. Calculate a 95 percent confidence interval for the “Gross output – Year 3 (Rs)” -- __To be assigned__
 2. Define two different measures that you consider most appropriate for measuring 
the performance of the units. This definition is up to you. These can be the 
variables that are already in the data or new variables defined based on the 
existing variables. For example, you can define a metric “Gross output per 
employee” by dividing the gross output of the year by the total number of 
employees. Please explain in one paragraph why you have selected these two 
measures and why you think they are most appropriate.[Solution](#id03) 
to be carried out based on these definitions.
 4. Calculate 99% confidence interval for the population mean for each of the two 
metrics defined by you. Interpret these confidence intervals in terms of their 
relevance to the management.[Solution](#id04)
 5.
   -  What is the probability that a firm selected at random is a SSSBE unit? -- __To be assigned__
   -  What is the probability that a firm selected at random is GOOD in 
performance? (Calculate the average of the first performance measure that 
you had defined in question 2 above. If the firm’s performance is above this 
average, it considered good. If it is below average, it is considered Bad)
   - What is the probability that a firm selected is a SSSBE Unit and ALSO 
GOOD in performance? 
   - What can you say about the performance of the SSSBE units in terms of 
GOOD or BAD based on the probabilities calculated above?
 6. Test the null hypothesis that the population average of the variable “Value of 
Exports for Year 3” = 87,300. Carry out a one-sided test. Clearly state your 
null and alternate hypotheses. -- __To be assigned__
 7. There is a feeling within the Central Government Department for promotion of 
small scale units, be it SSSBE or SSI, that if the population proportion is less 
than 25%, there is a need for providing special incentives. Based on your 
sample, would you recommend these special incentives for SSSBE or SSI or 
both?
 8. Some male chauvinists like to think that a larger proportion of SSSBEs are 
managed by men as compared to women. Do you agree with this contention? 
Explain your answer with appropriate statistical evidence.
 9. Based on the performance measures defined by you, is there a difference in 
performance between the SSSBE and SSI? If there is a difference, which of the 
two is better in performance?
 10. Is there a difference in the proportion of better performing units in the SSI and 
SSBE units? Use the following procedure to test this. Take one of the 
performance measures defined by you. Calculate the averages of this measure 
separately for SSI and SSSBE units. Define an SSI unit as better performing 
unit if the performance of the unit is more than the average of the SSI units. 
Calculate the proportion of better performing units of SSI. Calculate the 
proportion of higher performing units of SSSBE in similar manner. Use these 
two proportions to test the hypothesis. Repeat this with second measure defined 
by you. Comment on the results highlighting the differences, if any.
 11. Comment on the distribution of the two metrics (variables) created by you. Can 
you conclude that they follow normal distribution? Explain the rationale for your conclusion. -- __To be assigned__

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.stats import kstest,anderson
import scipy.stats as st
from pandas import DataFrame
from typing import Any

#### Functions Used in the notebook

In [2]:
def loadData(FilePath:str)-> Any:
    try:
        dataDf=pd.read_csv(FilePath)
        return dataDf
    except Exception as e:
        print(f"Facing below error {str(e)}")
        return None

def detailsOfDataFrame(srcDF:DataFrame):
    colLst=srcDF.columns
    print(f"The Shape of dataframe is {srcDF.shape}")
    print(f"The List of columns are {colLst}")
    # print(pd.DataFrame(srcDF.describe()).T.head())

def filtrData(srcDF:DataFrame,reqCols)-> Any:
    try:
        return srcDF[reqCols].copy()
    except Exception as e:
        print(f"There is an error in running this function {str(e)}")
        return None

def dataPreProcessing(sourceDF:DataFrame)->DataFrame:
    '''
    This function prepares four key ratio on sourceDataFrame
    args:
        sourceDF - Pandas DataFrame
    return:
        sourceDF - Pandas DataFrame
    '''
    try:
        # UNIT_TYPE=1 -- SSI else SSBE
        sourceDF['UnitTypeDiscr']= np.where(sourceDF['UNIT_TYPE']==1, \
                                                "SSI","SSBE")
        # CAGR = (FinalYear/InitialYear )**1/n -1
        sourceDF['CAGR'] = np.where(sourceDF['GOP_Year1'] ==0, \
                                sourceDF['GOP_Year1'], \
                                (np.power((sourceDF['GOP_Year3']/sourceDF['GOP_Year1']),1/3)-1)*100)
        # Asset Yeild Ratio = Gross Output Year3/ Original Purchase Value of Plant and Machinery
        sourceDF['AssetYieldRatio'] = np.where(sourceDF['ORI_PURC_VAL_PM'] ==0 \
                                            ,sourceDF['ORI_PURC_VAL_PM'], \
                                            (sourceDF['GOP_Year3']/sourceDF['ORI_PURC_VAL_PM']))
        #AssetApreciationRation = Market Value of Firm/Original Value of Plant and Machiner
        sourceDF['AssetApprPer'] = np.where(sourceDF['ORI_PURC_VAL_PM'] ==0, \
                                           sourceDF['ORI_PURC_VAL_PM'], \
                                           ((sourceDF['MKT_VAL_FA']/sourceDF['ORI_PURC_VAL_PM'])-1))
        #NetworthRatio = 3rd year Networth/2nd Year Networth
        sourceDF['NetWorthRatio'] = np.where(sourceDF['NET_Year2'] ==0, \
                                          sourceDF['NET_Year2'], \
                                          (sourceDF['NET_Year3']/sourceDF['NET_Year2']-1))
        print("CAGR, AssetYieldRatio, Asset Appr Ratio, NetworthRatio has been prepared")
    except Exception as e:
        print(f"There is an error in preparing ratios {str(e)}")
    return sourceDF

def genSmplsData(srcDF:DataFrame,NoOfSubSmpls=70,NoOfSmpls=1000)-> DataFrame:
    '''
    This function helps to generate Sample Data
    args:
        srcDf: Pandas DataFrame 
        NoOfSunSmpls: Integer, No of Sub Samples to be made
        NoOfSmpls: Integer, No of Sample to be made
    return:
        Pandas DataFrame
    '''
    keyRatioLst=[]
    for i in range(NoOfSubSmpls):
        srcDFSmpl= srcDF.sample(NoOfSmpls,random_state=i)
        keyRatioLst.append({'CAGR':srcDFSmpl['CAGR'].mean(), \
                         'AssetYieldRatio':srcDFSmpl['AssetYieldRatio'].mean(),
                           'AssetApprPer':srcDFSmpl['AssetApprPer'].mean(),
                           'NetWorthRatio':srcDFSmpl['NetWorthRatio'].mean(),
                           'GOP_Year3':srcDFSmpl['GOP_Year3'].mean()})
    return pd.DataFrame(data=keyRatioLst)

def addGoodCate(srcDF,paramCol):
    '''
    This function prepares new category column called Category
    args:
        srcDF: Pandas DataFrame
        paramCol: Column name on this category is prepared
    return:
        srcDF: Pandas DataFrame
    '''
    srcDF['Category']=np.where(srcDF[paramCol]>srcDF[paramCol].mean(), \
                                "GOOD", "BAD")
    return srcDF
    
    
def anderSonTest(dataTest:DataFrame,colLst:list):
    '''
    This function helps to do AnderSon Test reference code and content are 
    taken from https://pyshark.com/test-for-normality-using-python/
    '''
    # alpha=0.0125
    for coln in colLst:
        result=kstest(dataTest[coln],cdf='norm')
        resultAnderson = (anderson(dataTest[coln], dist='norm'))
        print("𝐻0: The data comes from a Normal distribution")
        print("𝐻1: The data doesn’t come from a Normal distribution") 
        print("\nResults of Anderson Test:\n")
        if resultAnderson[0] < resultAnderson[1][3]:
            print(f"Fail to reject null Hypothesis, hence {coln} follows normal distribution at {100-resultAnderson[2][3]*2}% confidence level ")
        else:
            print(f"Reject null Hypothesis, hence {coln} not follows normal distribution at {100-resultAnderson[2][3]*2}% confidence level")
        print("="*25)

#### Parameters to be defined in Notebook

In [3]:
dataFilePath="../null_values_removed_df.csv"
anlysisCols= ['UNIT_TYPE','MKT_VAL_FA', 'ORI_PURC_VAL_PM','GOP_Year3', \
                   'GOP_Year2', 'GOP_Year1', 'VOE_Year3','NET_Year2','NET_Year3']
NOOFSUBSAMPLES=70
NOOFSAMPLES=1000

#### <a id="id01"/> Data Loading & Preprocessing

In [4]:
NotNullDf=loadData(FilePath=dataFilePath)

In [5]:
detailsOfDataFrame(NotNullDf)

The Shape of dataframe is (10000, 66)
The List of columns are Index(['Unnamed: 0', 'Index', 'SL_NO', 'ST_CODE', 'DIST_CODE', 'TTM_CODE',
       'SECTOR_CODE', 'VT_CODE', 'WORKED_FOR_1D', 'CLOSED_Y', 'OPER_DUR_MM',
       'OPER_DUR_DD', 'SURVEY_CODE', 'NIC_CODE', 'PERM_REG_Y', 'FACTORY_ACT',
       'INIT_PROD_Y', 'UNIT_TYPE', 'ANCI_UNIT', 'ORG_TYPE', 'WOMAN_ENT',
       'MKT_VAL_FA', 'ORI_PURC_VAL_PM', 'PM_INST_Y', 'EMP_TOTAL', 'EMP_MALE',
       'EMP_FEMALE', 'EMP_CHILD', 'EMP_SC', 'EMP_ST', 'EMP_OBC', 'EMP_OTHER',
       'POWER_SRC', 'PROD1_CODE', 'PROD2_CODE', 'PROD3_CODE', 'PROD4_CODE',
       'PROD5_CODE', 'ACTI_NATURE', 'OPER_NATURE', 'MAN_BY', 'MAN_CASTE',
       'REG_UNDER_SPR', 'ACC_EXIST', 'COMP_EXIST', 'KNOW_HOW', 'GOP_Year3',
       'GOP_Year2', 'GOP_Year1', 'VOE_Year3', 'OS_LOAN_Year3', 'OS_INST_SRC',
       'OS_NONINST_SRC', 'NET_Year2', 'NET_Year3', 'NET_STATUS_Year2',
       'REPAYMENT_DELAY', 'DECLINE_IN_GOP', 'DMD_PROB', 'WORK_CAP_PROB',
       'RAW_MTRL_PROB', 'POWER_

In [6]:
NotNullDFReq=filtrData(NotNullDf,reqCols=anlysisCols)
NotNullDFReqRatio=dataPreProcessing(NotNullDFReq)
detailsOfDataFrame(NotNullDFReqRatio)

CAGR, AssetYieldRatio, Asset Appr Ratio, NetworthRatio has been prepared
The Shape of dataframe is (10000, 14)
The List of columns are Index(['UNIT_TYPE', 'MKT_VAL_FA', 'ORI_PURC_VAL_PM', 'GOP_Year3', 'GOP_Year2',
       'GOP_Year1', 'VOE_Year3', 'NET_Year2', 'NET_Year3', 'UnitTypeDiscr',
       'CAGR', 'AssetYieldRatio', 'AssetApprPer', 'NetWorthRatio'],
      dtype='object')


#### <a id="id03"/> Solution to Question02
 - Post understanding SSI & SSBE dataset, we came up with below list of performance metrics:
    - CAGR of 3 years
    - Asset Yield Ratio
    - Debt Ratio
    - MKT_VAL_FA
    - ORI_PURC_VAL_PM
    - EMP_TOTAL
    - GOP_Year3
    - VOE_Year3
    - NET_Year3
    - PercentageOfNetworthIncrease
 - Understandable, interpretable and usable?
 - Methodology:
  - Sampled 10000 Rows dataset and measure sampled means 70 samples
  - Applied Normal test (Anderson Test) on Sampled means of 70 datapoints of 6 Metrics 

In [7]:
# Prpearing Samples DataSet
smplDf=genSmplsData(NotNullDFReqRatio,NoOfSubSmpls=NOOFSUBSAMPLES,NoOfSmpls=NOOFSAMPLES)
nonTxcolLst=['CAGR','AssetYieldRatio','AssetApprPer','NetWorthRatio','GOP_Year3']
# Applying Normality Test
anderSonTest(smplDf,nonTxcolLst)

𝐻0: The data comes from a Normal distribution
𝐻1: The data doesn’t come from a Normal distribution

Results of Anderson Test:

Fail to reject null Hypothesis, hence CAGR follows normal distribution at 95.0% confidence level 
𝐻0: The data comes from a Normal distribution
𝐻1: The data doesn’t come from a Normal distribution

Results of Anderson Test:

Reject null Hypothesis, hence AssetYieldRatio not follows normal distribution at 95.0% confidence level
𝐻0: The data comes from a Normal distribution
𝐻1: The data doesn’t come from a Normal distribution

Results of Anderson Test:

Fail to reject null Hypothesis, hence AssetApprPer follows normal distribution at 95.0% confidence level 
𝐻0: The data comes from a Normal distribution
𝐻1: The data doesn’t come from a Normal distribution

Results of Anderson Test:

Reject null Hypothesis, hence NetWorthRatio not follows normal distribution at 95.0% confidence level
𝐻0: The data comes from a Normal distribution
𝐻1: The data doesn’t come from a Nor

##### From the above normality test results; we can conclude that _CAGR_ and _AssetAppreciation Ratio_ follows Normal Distirbution

In [8]:
smplDf.describe()

,CAGR,AssetYieldRatio,AssetApprPer,NetWorthRatio,GOP_Year3
count,70.000000,70.000000,70.000000,70.000000,7.000000e+01
mean,5.963213,305.182332,3.769821,0.178615,1.056203e+08
std,0.842872,831.253855,0.397117,0.098105,2.910259e+08
min,4.162393,5.330815,3.082348,0.045079,1.099230e+06
25%,5.297501,6.480399,3.493815,0.109586,1.614485e+06
50%,5.918618,7.461075,3.734546,0.144974,1.888215e+06
75%,6.572829,10.870171,4.044319,0.227973,2.190768e+06
max,8.088490,2612.356586,5.026495,0.456560,9.105518e+08


In [9]:
smplDfWCat=addGoodCate(smplDf,'CAGR')
detailsOfDataFrame(smplDfWCat)
smplDfWCat.head(10)

The Shape of dataframe is (70, 6)
The List of columns are Index(['CAGR', 'AssetYieldRatio', 'AssetApprPer', 'NetWorthRatio', 'GOP_Year3',
       'Category'],
      dtype='object')


,CAGR,AssetYieldRatio,AssetApprPer,NetWorthRatio,GOP_Year3,Category
0,6.109547,16.879418,3.475855,0.295905,2.157204e+06,GOOD
1,7.186105,18.386604,3.519364,0.165867,1.687495e+06,GOOD
2,5.841930,6.473149,3.749742,0.165221,1.735529e+06,BAD
3,5.824929,7.212214,3.921980,0.177697,1.650153e+06,BAD
4,5.996744,7.167700,4.142980,0.094462,2.555700e+06,GOOD
5,6.918361,2602.702538,3.700210,0.080997,9.099052e+08,GOOD
6,5.927023,7.621001,4.354992,0.310637,1.377312e+06,BAD
7,6.021926,2612.356586,3.723567,0.113828,9.100640e+08,GOOD
8,6.860057,7.320414,4.069430,0.115086,1.208911e+06,GOOD
9,5.733724,6.905085,3.430825,0.069825,2.405762e+06,BAD


#### <a id="id04"/> Calculation of 99% confidence interval of CAGR

In [10]:
meanCagr=smplDfWCat['CAGR'].mean()
stdErrOfMean=smplDfWCat['CAGR'].std()/np.sqrt(smplDfWCat['CAGR'].shape[0])
print(f"Mean and StdErrOfMean are f{meanCagr,stdErrOfMean}")
# Since the number of samples are 70 and this parameter follows normal distribution as per Anderson Hypothesis test
CI=99
lower99CI,upper99CI = st.norm.ppf(0.005,meanCagr,stdErrOfMean),st.norm.ppf(0.995,meanCagr,stdErrOfMean)
print(f"The values of 99% Confidence Interval lower and upper bound are {lower99CI,upper99CI}")

Mean and StdErrOfMean are f(5.963213390164637, 0.10074252545990292)
The values of 99% Confidence Interval lower and upper bound are (5.703717840971498, 6.222708939357776)


#### With confidence of 99% we can conclude that CAGR of a business will be in between 5.70% to 6.22%

In [11]:
smplDf.to_csv("PerformanceMetrics.csv",index=False)